# NLP

Предобработанный датафрейм подвергнем последующей обработке с помощью методов работы с естественным языком

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem.snowball import SnowballStemmer
import pymorphy2
from pymorphy2 import MorphAnalyzer

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_work = pd.read_csv(r'df_work.csv')

In [5]:
df_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896526 entries, 0 to 896525
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   OBJECT_NAME  896526 non-null  object 
 1   OKPD_NUM     896526 non-null  float64
dtypes: float64(1), object(1)
memory usage: 13.7+ MB


In [6]:
df_work.head()

,OBJECT_NAME,OKPD_NUM
0,Поставка металлической мебели,31.0
1,поставка офисных кресел,31.0
2,прочая закупка услуг (услуги по монтажу и пуск...,43.2
3,прочая закупка товара (стулья ученические),31.0
4,выполнение работ по текущему ремонту заполнени...,43.3


## Удаление СТОП-слов

Удаляем общеупотребительные слова (СТОП-СЛОВА), которые не помогают извлечь специфику данного предложения и символы, которые будут мешать дальнейшей обработке

In [7]:
STOPWORDS = set(stopwords.words('russian'))

In [8]:
def clean_text(text):
    """
    Функция для вычистки данных от латинских символов, цифр и перевода текста в нижний регистр
    """
    text = str(text).lower()  
    text = re.sub(r'[a-zA-Z0-9]', '', text)
    text = re.sub(r'[^\w\s]|_', ' ', text)
    return text

## Проводим токенизакцию и стемминг столбца OBJECT_NAME

In [9]:
def stemming(column, stopwords=STOPWORDS):
    stemmer = SnowballStemmer("russian")
    tokens_stem = [stemmer.stem(word) for word in word_tokenize(column) if not word in stopwords]
    return tokens_stem

In [10]:
def clean_column(df):
    """
    Обработка целевого столбца 
    """
    df['CLEAN_OBJECT_NAME'] = df['OBJECT_NAME'].apply(clean_text)
    df['STEM_OBJECT_NAME'] = df['CLEAN_OBJECT_NAME'].apply(
        lambda x: stemming(x, stopwords=STOPWORDS))
    return df

In [10]:
df_nlp = clean_column(df_work)
df_nlp

,OBJECT_NAME,OKPD_NUM,CLEAN_OBJECT_NAME,STEM_OBJECT_NAME
0,Поставка металлической мебели,31.0,поставка металлической мебели,"[поставк, металлическ, мебел]"
1,поставка офисных кресел,31.0,поставка офисных кресел,"[поставк, офисн, кресел]"
2,прочая закупка услуг (услуги по монтажу и пуск...,43.2,прочая закупка услуг услуги по монтажу и пуск...,"[проч, закупк, услуг, услуг, монтаж, пусконала..."
3,прочая закупка товара (стулья ученические),31.0,прочая закупка товара стулья ученические,"[проч, закупк, товар, стул, ученическ]"
4,выполнение работ по текущему ремонту заполнени...,43.3,выполнение работ по текущему ремонту заполнени...,"[выполнен, работ, текущ, ремонт, заполнен, око..."
...,...,...,...,...
896521,Поставка хозяйственных товаров для нужд ГБДОУ ...,13.9,поставка хозяйственных товаров для нужд гбдоу ...,"[поставк, хозяйствен, товар, нужд, гбдо, детск..."
896522,Поставка сантехники для ГБДОУ детского сада № ...,23.4,поставка сантехники для гбдоу детского сада ...,"[поставк, сантехник, гбдо, детск, сад, приморс..."
896523,Поставка мебели для пищеблока в ГБДОУ детский ...,25.9,поставка мебели для пищеблока в гбдоу детский ...,"[поставк, мебел, пищеблок, гбдо, детск, сад, п..."
896524,Поставка мебели для пищеблока в ГБДОУ детский ...,31.0,поставка мебели для пищеблока в гбдоу детский ...,"[поставк, мебел, пищеблок, гбдо, детск, сад, п..."


In [11]:
df_nlp.to_csv(r'df_nlp.csv', index=False)

Идея для дальнейшего исследования:

Выполнить не только стемминг столбца, но и лемматизацию, сохранить в отдельных столбцах в качестве разных данных X для последующей классификации и сравнить полученные результаты:

Который из методов обработки естественного языка окажется более подходящим для данной задачи

Лемматизацию проведем с помощью модуля pymorphy2:

In [11]:
def lemmatize(column, stopwords=STOPWORDS):
    morph = MorphAnalyzer()
    tokens_morph = [morph.normal_forms(word) for word in word_tokenize(column) if not word in stopwords]
    return tokens_morph

В качестве тестового набора данных возьмем небольшую часть исходного датафрейма - 5000 строк и построим нужные столбцы:

In [12]:
df_work_chunk = df_work.iloc[:5000,:]

In [13]:
def clean_column_stem_lem(df):
    """
    Обработка целевого столбца 
    """
    df['CLEAN_OBJECT_NAME'] = df['OBJECT_NAME'].apply(clean_text)
    df['STEM_OBJECT_NAME'] = df['CLEAN_OBJECT_NAME'].apply(
        lambda x: stemming(x, stopwords=STOPWORDS))
    df['MORPH_OBJECT_NAME'] = df['CLEAN_OBJECT_NAME'].apply(
        lambda x: lemmatize(x, stopwords=STOPWORDS))
    return df

In [14]:
df_npl_chunk = clean_column_stem_lem(df_work_chunk)
df_npl_chunk

,OBJECT_NAME,OKPD_NUM,CLEAN_OBJECT_NAME,STEM_OBJECT_NAME,MORPH_OBJECT_NAME
0,Поставка металлической мебели,31.0,поставка металлической мебели,"[поставк, металлическ, мебел]","[[поставка], [металлический], [мебель]]"
1,поставка офисных кресел,31.0,поставка офисных кресел,"[поставк, офисн, кресел]","[[поставка], [офисный], [кресло]]"
2,прочая закупка услуг (услуги по монтажу и пуск...,43.2,прочая закупка услуг услуги по монтажу и пуск...,"[проч, закупк, услуг, услуг, монтаж, пусконала...","[[прочий], [закупка], [услуга], [услуга], [мон..."
3,прочая закупка товара (стулья ученические),31.0,прочая закупка товара стулья ученические,"[проч, закупк, товар, стул, ученическ]","[[прочий], [закупка], [товар], [стул], [ученич..."
4,выполнение работ по текущему ремонту заполнени...,43.3,выполнение работ по текущему ремонту заполнени...,"[выполнен, работ, текущ, ремонт, заполнен, око...","[[выполнение], [работа], [текущий, течь], [рем..."
...,...,...,...,...,...
4995,Оказание услуг по техническому обслуживанию си...,41.2,оказание услуг по техническому обслуживанию си...,"[оказан, услуг, техническ, обслуживан, сист, в...","[[оказание], [услуга], [технический], [обслужи..."
4996,Оказание услуг по техническому обслуживанию ли...,43.2,оказание услуг по техническому обслуживанию ли...,"[оказан, услуг, техническ, обслуживан, лифтов,...","[[оказание], [услуга], [технический], [обслужи..."
4997,Выполнение проектно-изыскательских работ и раб...,42.1,выполнение проектно изыскательских работ и раб...,"[выполнен, проектн, изыскательск, работ, работ...","[[выполнение], [проектный], [изыскательский], ..."
4998,Поставка тележек для дворников ГБУ Жилищник ра...,25.9,поставка тележек для дворников гбу жилищник ра...,"[поставк, тележек, дворник, гбу, жилищник, рай...","[[поставка], [тележка], [дворник, дворников], ..."
